# Daily Filtering

In [1]:
from nselib import capital_market
import concurrent.futures
from tqdm import tqdm
import warnings
import numpy as np
import pandas as pd
from datetime import datetime
from swing_trading.nse import nse_self
nse = nse_self()
warnings.filterwarnings("ignore")

In [2]:
today = '30-06-2025'

In [3]:
today_stocks_info = nse.get_nse_equities_by_date(today)
stocks = today_stocks_info[today_stocks_info['SERIES'] == 'EQ']['SYMBOL'].tolist()

In [4]:
df.loc[
    df['SYMBOL'] == 'SSWL', ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE','TTL_TRD_QNTY']
    ].rename(columns={
        'OPEN_PRICE': 'O',
        'HIGH_PRICE':'H',
        'LOW_PRICE': 'L',
        'CLOSE_PRICE': 'C',
        'TTL_TRD_QNTY': 'Volume'
    })

,SYMBOL,O,H,L,C,Volume
2498,SSWL,271.31,277.56,271.31,274.0,631340


In [5]:
nse.get_nse_stock_by_duration('CUB', '1M')

,Symbol,Series,Date,PrevClose,OpenPrice,HighPrice,LowPrice,LastPrice,ClosePrice,AveragePrice,TotalTradedQuantity,TurnoverInRs,No.ofTrades,DeliverableQty,%DlyQttoTradedQty,"ï»¿""Symbol"""
0,NaN,EQ,30-May-2025,194.34,194.30,197.00,192.40,195.00,195.70,194.96,2263850,4.413713e+08,41400,1343802,59.36,CUB
1,NaN,EQ,02-Jun-2025,195.70,195.00,199.60,193.32,197.64,198.64,197.59,4406952,8.707505e+08,49424,3099967,70.34,CUB
2,NaN,EQ,03-Jun-2025,198.64,199.50,204.00,197.70,201.20,202.28,202.01,6116892,1.235664e+09,111502,4148817,67.83,CUB
3,NaN,EQ,04-Jun-2025,202.28,203.99,203.99,197.11,197.56,197.76,199.71,1355968,2.707952e+08,32869,732779,54.04,CUB
4,NaN,EQ,05-Jun-2025,197.76,196.85,199.50,195.81,197.85,198.71,198.14,1304782,2.585337e+08,36720,685067,52.50,CUB
5,NaN,EQ,06-Jun-2025,198.71,199.99,202.12,198.00,202.00,201.17,200.19,1336958,2.676455e+08,24938,740792,55.41,CUB
6,NaN,EQ,09-Jun-2025,201.17,204.00,205.00,201.00,204.65,204.67,203.53,2854330,5.809529e+08,40426,1960938,68.70,CUB
7,NaN,EQ,10-Jun-2025,204.67,205.05,208.40,205.05,206.00,206.27,206.82,1987644,4.110804e+08,28863,1200498,60.40,CUB
8,NaN,EQ,11-Jun-2025,206.27,206.27,206.50,199.50,200.00,199.94,201.68,1468060,2.960812e+08,27028,890884,60.68,CUB
9,NaN,EQ,12-Jun-2025,199.94,200.88,202.92,194.50,195.50,195.18,197.77,1157653,2.289513e+08,19797,592302,51.16,CUB


# level 1 - MA 200 & Vol 2.25x

In [8]:
filtered_stocks_1 = []

def process_stock(stock):
    try:
        data = nse.get_nse_stock_by_duration(symbol=stock, period='1Y')
    except:
        # skipping; stock with trades less than 1000 TDs
        return None


    # -- MA 200 -- 
    MA = 200
    close_series = data[::-1]['ClosePrice'][:MA]
    if close_series.empty or len(close_series.dropna()) == 0:
        return None 
    
    close = close_series.replace({',': ''}, regex=True).astype(float)
    if len(close) < MA:
        return None

    closePrice = close.iloc[0]
    MA_200 = close.mean()

    # TODO: considering adding some buffer
    if closePrice < MA_200:
        # NOTE: skipping; stock has been listed for less than 200 days
        return None


    # -- Volume > 2.25x 10D Avg --
    try:
        volume_data = capital_market.deliverable_position_data(stock, period='1M')['TradedQty']
        volume = volume_data[::-1].str.replace(',', '', regex=False).astype(int)
    except:
        return None

    current_vol = volume.iloc[0]
    # volume needs to above 100k
    if current_vol > 100_000:
        last_10_days_avg = volume.iloc[1:11].mean()
        if current_vol > last_10_days_avg * 2.25:
            return stock

    return None


print(f'Total no. of stocks: {len(stocks)}')

with concurrent.futures.ThreadPoolExecutor(max_workers=11) as executor:
    futures = [executor.submit(process_stock, stock) for stock in stocks]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Flitering'):
        result = f.result()
        if result:
            filtered_stocks_1.append(result)

print(f'Filtered stocks: {len(filtered_stocks_1)}')

Total no. of stocks: 2045


Flitering: 100%|██████████| 2045/2045 [11:38<00:00,  2.93it/s]

Filtered stocks: 77


# level 2 - excluding ETFs

In [9]:
exclusions = ["ETF", "GOLD", "SILVER", "NIF100BEES", "NIFTY"]
excluded_stocks = []
filtered_stocks_2 = []

for stk in filtered_stocks_1:
    if any(excl in stk for excl in exclusions):
        excluded_stocks.append(stk)
    else:
        filtered_stocks_2.append(stk)

print(f"Excluded stocks: {excluded_stocks}")
len(filtered_stocks_2)

Excluded stocks: ['AUTOIETF', 'BSLNIFTY', 'GOLDTECH', 'MIDCAPIETF', 'MIDCAPETF', 'PSUBNKIETF']


71

# level 3 - higher opens & higher closes

In [18]:
filtered_stocks_3 = []

def check_price_movement(stock):
    try:
        # get lastest 7 days of data
        data = nse.get_nse_stock_by_duration(symbol=stock, period='1W')
        # latest
        data = data[::-1]  
        close_prices = data['ClosePrice'].replace({',': ''}, regex=True).astype(float)
        open_prices = data['OpenPrice'].replace({',': ''}, regex=True).astype(float)

        if len(close_prices) >= 2 and len(open_prices) >= 1:
            today_close = close_prices.iloc[0]
            prev_close = close_prices.iloc[1]
            today_open = open_prices.iloc[0]
            prev_open = open_prices.iloc[1]

            # Condition: today's close > previous close AND today's close > open
            if today_close > prev_close and today_close > today_open and today_open > prev_open:
                return stock
    except Exception as e:
        return None
    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(check_price_movement, stock) for stock in filtered_stocks_2]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Price Check'):
        result = f.result()
        if result:
            filtered_stocks_3.append(result)

# sorting alphabetically
filtered_stocks_3.sort()
print(f"Filtered stocks: {len(filtered_stocks_3)}")
print(filtered_stocks_3)

Price Check:   0%|          | 0/71 [00:00<?, ?it/s]

Price Check: 100%|██████████| 71/71 [00:05<00:00, 12.79it/s]

Filtered stocks: 42
['APEX', 'APOLLOHOSP', 'ASAHIINDIA', 'BANCOINDIA', 'CASTROLIND', 'CENTEXT', 'CREDITACC', 'CUB', 'DANGEE', 'ENDURANCE', 'EQUITASBNK', 'FAZE3Q', 'FILATEX', 'FMGOETZE', 'GILT5YBEES', 'GNFC', 'HBLENGINE', 'IDFCFIRSTB', 'INDNIPPON', 'INNOVANA', 'JKLAKSHMI', 'KARURVYSYA', 'KOLTEPATIL', 'LAURUSLABS', 'LORDSCHLO', 'MAHABANK', 'NITCO', 'PFOCUS', 'PROZONER', 'PSPPROJECT', 'PSUBANK', 'PTL', 'RUSTOMJEE', 'SHREDIGCEM', 'SHREEPUSHK', 'SINCLAIR', 'SMLISUZU', 'SUPRIYA', 'TNTELE', 'VETO', 'VISAKAIND', 'YATHARTH']


# level 4 - previous 10 days's daily vol > 100k

In [22]:
filtered_stocks_4 = []

def check_volume_consistency(stock):
    try:
        # Get last 15 days of data (to ensure we have at least 10 trading days)
        volume_data = nse.get_nse_stock_by_duration(symbol=stock, period='1M')['TotalTradedQuantity']
        volume = volume_data[::-1]

        # Take last 10 trading days
        last_10_vols = volume.iloc[:10]

        # Check if all volumes > 100k
        if len(last_10_vols) < 10:
            return None

        if all(v > 100_000 for v in last_10_vols):
            return stock
    except:
        return None
    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(check_volume_consistency, stock) for stock in filtered_stocks_3]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Volume Consistency'):
        result = f.result()
        if result:
            filtered_stocks_4.append(result)

# Sort alphabetically
filtered_stocks_4.sort()
print(f"Filtered Stocks_4: {len(filtered_stocks_4)}")
print(filtered_stocks_4)

Volume Consistency:   0%|          | 0/42 [00:00<?, ?it/s]

Volume Consistency: 100%|██████████| 42/42 [00:02<00:00, 14.01it/s]

Filtered Stocks_4: 21
['APOLLOHOSP', 'ASAHIINDIA', 'BANCOINDIA', 'CASTROLIND', 'CREDITACC', 'CUB', 'ENDURANCE', 'EQUITASBNK', 'FILATEX', 'GILT5YBEES', 'GNFC', 'HBLENGINE', 'IDFCFIRSTB', 'JKLAKSHMI', 'KARURVYSYA', 'KOLTEPATIL', 'LAURUSLABS', 'MAHABANK', 'PFOCUS', 'PSPPROJECT', 'YATHARTH']


# -- WIP -- (to be level 5)

# final

In [23]:
filtered_stocks_4.sort()
np.array(filtered_stocks_4)

array(['APOLLOHOSP', 'ASAHIINDIA', 'BANCOINDIA', 'CASTROLIND',
       'CREDITACC', 'CUB', 'ENDURANCE', 'EQUITASBNK', 'FILATEX',
       'GILT5YBEES', 'GNFC', 'HBLENGINE', 'IDFCFIRSTB', 'JKLAKSHMI',
       'KARURVYSYA', 'KOLTEPATIL', 'LAURUSLABS', 'MAHABANK', 'PFOCUS',
       'PSPPROJECT', 'YATHARTH'], dtype='<U10')

# - 

In [16]:
for stk in filtered_stocks_4:
    result = today_stocks_info.loc[
        today_stocks_info['SYMBOL'] == stk,
        ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE']
    ]
    
    print(result.values.tolist())
    print()

[['APOLLOHOSP', 7336.5, 7336.5, 7203.5, 7242.0]]

[['BANCOINDIA', 599.4, 648.1, 596.0, 641.0]]

[['CUB', 208.5, 221.0, 205.1, 218.71]]

[['ENDURANCE', 2663.3, 2790.0, 2645.6, 2770.1]]

[['EQUITASBNK', 66.94, 68.24, 66.78, 67.11]]

[['FILATEX', 54.4, 55.81, 54.4, 55.21]]

[['GILT5YBEES', 62.32, 62.5, 62.29, 62.39]]

[['GNFC', 550.1, 557.05, 549.2, 553.0]]

[['HBLENGINE', 593.2, 600.5, 583.65, 592.9]]

[['IDFCFIRSTB', 72.38, 73.35, 72.21, 72.84]]

[['JKLAKSHMI', 886.25, 905.85, 886.25, 899.45]]

[['KARURVYSYA', 259.92, 268.8, 259.0, 267.54]]

[['KOLTEPATIL', 461.9, 470.0, 460.45, 464.5]]

[['LAURUSLABS', 709.8, 727.0, 704.0, 724.85]]

[['MAHABANK', 54.6, 57.35, 54.6, 57.11]]

[['PFOCUS', 137.99, 141.19, 131.0, 138.83]]

[['PSPPROJECT', 758.5, 765.0, 752.0, 759.9]]

[['YATHARTH', 528.45, 538.65, 528.0, 535.25]]



In [ ]:
[['ABCAPITAL', 246.0, 251.0, 244.52, 246.25]]

[['BLISSGVS', 137.71, 159.8, 137.71, 152.7]]

[['BLSE', 222.79, 232.5, 220.4, 225.13]]

[['CREDITACC', 1216.6, 1249.0, 1205.0, 1221.8]]

[['DHANI', 62.44, 72.59, 62.13, 71.09]]

[['GTL', 9.35, 11.18, 9.35, 11.18]]

[['INDSWFTLAB', 96.4, 114.38, 96.37, 110.27]]

[['INTELLECT', 1185.0, 1239.0, 1182.3, 1215.1]]

[['JPPOWER', 18.07, 19.3, 17.77, 18.22]]

[['MAHSEAMLES', 699.0, 723.9, 693.05, 719.35]]

[['MARKSANS', 252.15, 270.7, 252.0, 263.55]]

[['NATHBIOGEN', 194.95, 208.0, 194.85, 203.9]]

[['NIACL', 194.59, 208.49, 194.03, 198.92]]

[['PFS', 41.0, 44.19, 40.91, 43.6]]

[['PRSMJOHNSN', 152.0, 162.0, 152.0, 159.97]]

[['PTC', 178.0, 183.3, 176.38, 179.91]]

[['RAIN', 148.19, 156.79, 147.36, 154.98]]

[['RTNINDIA', 60.13, 67.5, 60.1, 65.56]]

[['RTNPOWER', 14.52, 16.13, 14.45, 15.24]]

[['SEQUENT', 189.1, 203.45, 189.1, 199.85]]

[['TRIDENT', 31.64, 33.14, 31.47, 31.99]]

[['TTML', 72.0, 81.12, 71.49, 75.85]]

[['WOCKPHARMA', 1555.0, 1847.8, 1545.0, 1831.1]]

# 9.15 to 9.30

In [ ]:
import requests
import time
from datetime import datetime

In [ ]:
api_key = '2dabbfcdf93040ba99894559f7a27ac5'
symbol = 'RELIANCE.NSE'

base_url = 'https://api.twelvedata.com/time_series'

# time window: 9.15 to 9.30
start_hour = 9
start_minute = 15
end_minute = 30

while True:
    # current time
    now = datetime.now()
    
    # if inside the window
    if now.hour == start_hour and start_minute <= now.minute <= end_minute:
        params = {
            'symbol': symbol,
            'interval': '5min',
            'outputsize': 1,
            'apikey': api_key
        }
        response = requests.get(base_url, params=params)
        data = response.json()

        if 'values' in data:
            last_candle = data['values'][0]
            print(f"{last_candle['datetime']} - O:{last_candle['open']} H:{last_candle['high']} L:{last_candle['low']} C:{last_candle['close']}")
        else:
            print("Error or limit reached:", data)

        time.sleep(300)  # Wait 5 mins

    elif now.minute > end_minute:
        print("Finished 9:15–9:30 fetching.")
        break
    else:
        time.sleep(10)  # Before 9:15, check every 10 sec
